In [ ]:
pip install -U sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string
from common import RANDOM_SEED

In [ ]:
df_fake = pd.read_csv("Negative.csv", encoding='latin-1')
df_true = pd.read_csv("Positive.csv", encoding='latin-1')

In [ ]:
df_true.head(10)

In [ ]:
df_fake["class"] = 0
df_true["class"] = 1

In [ ]:
df_fake.shape, df_true.shape

In [ ]:
df_fake_manual_testing = df_fake.tail(50)
for i in range(23480,23430, -1):
    df_fake.drop([i], axis=0, inplace=True)
df_true_manual_testing = df_true.tail(50)
for i in range(21416,21366, -1):
    df_true.drop([i], axis=0, inplace=True)

In [ ]:
df_fake.shape, df_true.shape

In [ ]:
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis=0)
df_manual_testing.to_csv("manual_testing.csv")

In [ ]:
df_merge = pd.concat([df_fake, df_true], axis=0)
df_merge.head(10)

In [ ]:
df_merge = df_merge.sample(frac=1)

In [ ]:
df_merge.head(10)

In [ ]:
df_merge.isnull().sum()

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [ ]:
df_merge["text"] = df_merge["text"].apply(wordopt)

In [ ]:
df_merge.head(10)

In [ ]:
x = df_merge["text"]
y = df_merge["class"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state= RANDOM_SEED)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train.values.astype('U'))
xv_test = vectorization.transform(x_test.values.astype('U'))

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
xv_train1 = model.encode(x_train.values.astype('U'))
xv_test1 = model.encode(x_test.values.astype('U'))

##### Logistic Regression TF-IDF

In [92]:
from sklearn.linear_model import LogisticRegression

In [93]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [94]:
LR.score(xv_test, y_test)

0.8994488671157379

In [95]:
pred_lr=LR.predict(xv_test)

In [96]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90     11932
           1       0.90      0.91      0.90     12563

    accuracy                           0.90     24495
   macro avg       0.90      0.90      0.90     24495
weighted avg       0.90      0.90      0.90     24495



#### Logistic Regression Sbert

In [97]:
from sklearn.linear_model import LogisticRegression

In [98]:
LR = LogisticRegression()
LR.fit(xv_train1,y_train)

C:\Users\Jose Gabriel\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [99]:
LR.score(xv_test1, y_test)

0.8061645233721167

In [100]:
pred_lr=LR.predict(xv_test1)

In [101]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80     11932
           1       0.81      0.81      0.81     12563

    accuracy                           0.81     24495
   macro avg       0.81      0.81      0.81     24495
weighted avg       0.81      0.81      0.81     24495



##### Decision Tree

In [102]:
from sklearn.tree import DecisionTreeClassifier

In [103]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [104]:
pred_dt = DT.predict(xv_test)

In [105]:
DT.score(xv_test, y_test)

0.7425597060624617

In [106]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.73      0.75      0.74     11932
           1       0.75      0.74      0.75     12563

    accuracy                           0.74     24495
   macro avg       0.74      0.74      0.74     24495
weighted avg       0.74      0.74      0.74     24495



#### Decision Tree Sbert

In [107]:
from sklearn.tree import DecisionTreeClassifier

In [108]:
DT = DecisionTreeClassifier()
DT.fit(xv_train1, y_train)

DecisionTreeClassifier()

In [109]:
pred_dt = DT.predict(xv_test1)

In [110]:
DT.score(xv_test1, y_test)

0.6785466421718718

In [111]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.66      0.69      0.68     11932
           1       0.69      0.67      0.68     12563

    accuracy                           0.68     24495
   macro avg       0.68      0.68      0.68     24495
weighted avg       0.68      0.68      0.68     24495



##### Gradient Boosting

In [112]:
from sklearn.ensemble import GradientBoostingClassifier

In [113]:
GBC = GradientBoostingClassifier(random_state=1)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=1)

In [114]:
pred_gbc = GBC.predict(xv_test)

In [115]:
GBC.score(xv_test, y_test)

0.8120840987956726

In [116]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.84      0.76      0.80     11932
           1       0.79      0.87      0.83     12563

    accuracy                           0.81     24495
   macro avg       0.82      0.81      0.81     24495
weighted avg       0.81      0.81      0.81     24495



#### Gradient Boosting Sbert

In [117]:
from sklearn.ensemble import GradientBoostingClassifier

In [118]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train1, y_train)

GradientBoostingClassifier(random_state=0)

In [119]:
pred_gbc = GBC.predict(xv_test1)

In [120]:
GBC.score(xv_test1, y_test)

0.7886507450500102

In [121]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.79      0.78      0.78     11932
           1       0.79      0.80      0.79     12563

    accuracy                           0.79     24495
   macro avg       0.79      0.79      0.79     24495
weighted avg       0.79      0.79      0.79     24495



##### Random Forest Classifier 

In [122]:
from sklearn.ensemble import RandomForestClassifier

In [123]:
RFC = RandomForestClassifier(random_state=1)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=1)

In [124]:
pred_rfc = RFC.predict(xv_test)

In [125]:
RFC.score(xv_test, y_test)

0.8576444172280057

In [126]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.89      0.81      0.85     11932
           1       0.83      0.90      0.87     12563

    accuracy                           0.86     24495
   macro avg       0.86      0.86      0.86     24495
weighted avg       0.86      0.86      0.86     24495



#### Random Forest Classifier Sbert 

In [127]:
from sklearn.ensemble import RandomForestClassifier

In [128]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train1, y_train)

RandomForestClassifier(random_state=0)

In [129]:
pred_rfc = RFC.predict(xv_test1)

In [130]:
RFC.score(xv_test1, y_test)

0.7988977342314758

In [131]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.83      0.74      0.78     11932
           1       0.78      0.85      0.81     12563

    accuracy                           0.80     24495
   macro avg       0.80      0.80      0.80     24495
weighted avg       0.80      0.80      0.80     24495



#### AdaBoosting 

In [81]:
from sklearn.ensemble import AdaBoostClassifier

In [82]:
ADB= AdaBoostClassifier(random_state=1)
ADB.fit(xv_train, y_train)

AdaBoostClassifier(random_state=1)

In [83]:
pred_ADB = ADB.predict(xv_test)

In [84]:
ADB.score(xv_test, y_test)

0.7983670136762605

In [85]:
print(classification_report(y_test, pred_ADB))

              precision    recall  f1-score   support

           0       0.81      0.76      0.79     11932
           1       0.79      0.83      0.81     12563

    accuracy                           0.80     24495
   macro avg       0.80      0.80      0.80     24495
weighted avg       0.80      0.80      0.80     24495



#### Adaboosting Sbert

In [87]:
from sklearn.ensemble import AdaBoostClassifier

In [88]:
ADB= AdaBoostClassifier(random_state=1)
ADB.fit(xv_train1, y_train)

AdaBoostClassifier(random_state=1)

In [89]:
pred_ADB = ADB.predict(xv_test1)

In [90]:
ADB.score(xv_test1, y_test)

0.7552969993876302

In [91]:
print(classification_report(y_test, pred_ADB))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75     11932
           1       0.76      0.76      0.76     12563

    accuracy                           0.76     24495
   macro avg       0.76      0.76      0.76     24495
weighted avg       0.76      0.76      0.76     24495

